# Joomla 3

Con esta guía de Jupyter vamos a lograr instalar una versión reciente de Joomla, en un servidor Ubuntu 16.04 recién instalado.

Primero debe instalar los componentes del servidor LAMP, inciciando por la instalación de la base de datos. Seguidamente descargamos el código de Joomla.

Por último, creamos la base de datos a ser utilizada por Joomla, y luego corremos el instalador Web.


### MariaDB
MySQL y MariaDB tienen algunas pequeñas diferencias pero para nuestro objetivo son técnicamente equivalente, por lo cual vamos a preferir MariaDB sobre MySQL.

A la hora de instalar la base de datos de MariaDB, la herramienta de Apt de Debian va a insistir en solicitarnos una contraseña para el usuario "root" de la base de datos, el cual vamos a utilizar para tareas de administración.

Para evitar que en esta guía se nos solicite contraseña, vamos a usar el comando ``debconf-set-selections`` para definir la contraseña de antemano.

Nota: El usuario "root" de MariaDB y el de Linux son completamente aparte.

In [ ]:
echo "mariadb-server mysql-server/root_password password greencore" | sudo debconf-set-selections
echo "mariadb-server mysql-server/root_password_again password greencore" | sudo debconf-set-selections
sudo apt-get -y install mariadb-server

### Apache y Php

Procedemos ahora a instalar Apache2, con el componente de Php el cual fuerza el servidor a usar el MPM **prefork**.

Así mismo instalamos módulos de Php requeridos por la aplicación Joomla, según la documentación del fabricante.

Nota: No vamos a reiniciar el servicio de Apache en este momento, sino hasta luego de editar archivos de configuración.

In [ ]:
sudo apt-get -y install php php-mysql php-curl php-json php-cgi php-mcrypt php-xml libapache2-mod-php apache2 php-zip
sudo a2enmod rewrite

### Cambios menores en Apache

**Desde una terminal**, y utilizando su editor preferido, debe cambiar dos archivos de la siguiente forma:

**Archivo:** ``/etc/apache2/mods-enabled/dir.conf``

**Cambio:** La línea que inicia por DirectoryIndex tiene una lista de las extensiones de archivos de tipo "Index". Ya sea cambiando el orden, o eliminando extensiones, debe asegurarse que ``index.php`` esté de primero en la lista.

``DirectoryIndex index.php``

**Archivo:** ``/etc/apache2/apache2.conf``

**Cambio:** En la definición del directorio ``/var/www`` originalmente tiene la instrucción ``AllowOverride None``. Cambie la definición para que diga ``AllowOverride All``

``<Directory /var/www/>
        Options Indexes FollowSymLinks
        AllowOverride All
        Require all granted
</Directory>``

### Cambios menores en Php

**Archivo:** ``/etc/php/7.0/apache2/php.ini``

**Cambio:** Joomla recomienda para mejor desempeño, desactivar en Php la función de Output Buffering. Para ello vamos a reemplazar el texto ``output_buffering: 4096`` para que se muestre de la siguiente forma:

``
    output_buffering: off
``

Reiniciamos Apache y mostramos el estado del servicio.

## Joomla

La última versión y recomendada de Joomla es la 3.7.4. Sin embargo vamos a instalar una versión anterior, para practicar procesos de actualización.

In [ ]:
sudo systemctl restart apache2
sudo systemctl status apache2 | cat
wget -qc https://github.com/joomla/joomla-cms/releases/download/3.7.3/Joomla_3.7.3-Stable-Full_Package.tar.bz2
ls -lh

Descomprimimos el código de Joomla dentro de /var/www/html que es el directorio default de Apache.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para Joomla y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Apache. Esto porque Joomla debe poder manipular archivos dentro de la carpeta.

In [ ]:
sudo rm -v /var/www/html/index.html
sudo tar xjf ~/Joomla_3.7.3-Stable-Full_Package.tar.bz2 -C /var/www/html/
sudo chown -R www-data:www-data /var/www/html
ls /var/www/html

### Creación de base de datos

Antes de ejecutar el instalador Web de Joomla debemos crear la base de datos y el usuario que usará la aplicación de Joomla.

Para ello vamos a utilizar el comando de mysql(1), definiendo la contraseña "greencore" en la misma línea de comando.

In [ ]:
sudo mysql -u root -pgreencore << EOF
CREATE DATABASE joomla;
CREATE USER 'joomla'@'localhost' IDENTIFIED BY 'greencore';
GRANT ALL PRIVILEGES ON joomla.* TO 'joomla'@'localhost' IDENTIFIED BY 'greencore' WITH GRANT OPTION;
FLUSH PRIVILEGES;
EOF

### Instalador web

No es necesario manipular ningún archivo Php para definir la configuración de nuestro sitio. Visitamos el sitio web, donde nos recibe el instalador Web de Joomla. Este funciona en estilo "wizard" donde simplemente vamos contestando con cuidado las consultas que nos hace.

http://10.42.25.5X/


En "Welcome" puede usar valores que usaría en un sitio en producción.
Recuerde utilizar para este laboratorio el usuario "admin", con contraseña "greencore" para facilitar la revisión de errores o dudas.

Se recomienda para este ejemplo, durante el Wizard de instalación selecionar alguna opción en "**Install Sample Data**", coordinando de forma que los estudiantes puedan probar diversos ejemplos.